<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# [Ateliers: Technologies des grosses data](https://github.com/wikistat/Ateliers-Big-Data)

# Trafic de données (*data munging*) avec <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/> </a>

**Résumé**: L'objectif de ce tutoriel est d'introduire les objets de la technologie [Spark](https://spark.apache.org/) et leur utilisation à l'aide de commandes en Python, plus précisément en utilisant l'API  [`PySpark`](http://spark.apache.org/docs/latest/api/python/). Motivations à l'utilisation de cet environnement qui distribue automatiquement les données sur un cluster et parallélise les tâches; description des principaux types de données et du concept de *Resilient Distributed Datasets* (RDD): toute tâche en *Spark* s'exprime comme la création, la transformation de RDDs puis le lancement d'actions sur ces RDDs. 

## 1 Pourquoi Spark

Les données sont trop volumineuses pour la mémoire (RAM, disque) et/ou les temps de calcul rédhibitoires. Plus précisément, la distribution des données avec une technologies *Hadoop* est déjà en place ou devient incontournable. La description d'Hadoop n'est pas détaillée, non plus que les principes contraignants des fonctionnalités *MapReduce* associées. Consulter [Besse et al. 2016](https://hal.archives-ouvertes.fr/hal-01350099v2/document) pour une introduction à *MapReduce*, ses contraintes et une dicussion sur l'intérêt ou non de déployer Spark en fonction des traitements à mettre en oeuvre.

*Rappelons le contexte:*
- Les méthodes d'apprentissage statistique supervisée ou non déploient des algorithmes itératifs dont l'exécution sur des données gérées dans un cadre *Hadoop* provoquent des lectures / écritures à chaque itération. Les temps d'exécution s'en trouvent fortement pénalisés.
- La technologie [Spark](http://spark.apache.org/) y remédie en intégrant le concept de *tables de données distribuées résilientes* (*Resilient Distributed Dataset* ou RDD de Zaharia et al. 2012). Très schématiquement, chaque partition de données reste en mémoire sur son serveur de calcul entre deux itérations tout en gérant les principes de tolérance aux pannes propres à Hadoop.
- Les commandes spécifiques de *Spark*  s'exécutent en Java, Scala et aussi pour certaines en Python ou R. D'où l'intérêt de l'apprentissage de Python qui permet sans "trop" d'efforts de franchir le changement d'échelle en volume et vélocité, notamment par l'emploi de la librairie ou plutôt de l'interface de programmation (*Application Interface programmation* ou API) dédiée [`PySpark`](http://spark.apache.org/docs/latest/api/python/); cette API intère un noyau Jupyter pour une utilisation interactive sous forme de calepins.
- [SparlSQL](https://spark.apache.org/sql/) permet d'accéder à tout type de données, structurées ou non, volumineuse (distribuées) en Python exécutant des commandes dans une syntaxe SQL.
- [`MLlib`](http://spark.apache.org/docs/latest/ml-guide.html) intègre les principaux algorithmes d'apprentissage et méthodes statistique.
- Deux autres librairies sont disponibles pour traiter des données en [flux](https://spark.apache.org/streaming/)  (*streaming*) ou celles de [graphes et réseaux](https://spark.apache.org/graphx/). Elles ne sont actuellement (version 1.6 de Spark) accessibles que par les langages Scala ou Java.

Une évolution de *MLlib* est en cours à partir de la version 2. Elle permet d'intégrer progressivement les notions de *data frame* et *pipeline* aux algorithmes d'apprentissage machine déjà implémentées dans *MLlib* et applicables à des RDDs.

 
La principale motivation pour utiliser Spark est que les mêmes programmes ou commandes sont utilisées sur un poste isolé,  un cluster, un ensemble de machines virtuelles sur un serveur comme *Amazon Web Service*, *Google cloud*..., pour gérer prétraiter des données stockées dans un fichier ou distribuées dans un système *Hadoop*. 

**Attention**: (cf. [Besse et al. 2016](https://hal.archives-ouvertes.fr/hal-01350099v2/document)) Pour la phase d'apprentissage ou d'estimation de modèles, une solution qui consisterait à utiliser un plus gros ordinateur avec beaucoup de mémoire vive (architecture intégrée) est souvent  préférable en terme de temps de mise au point et d'exécution, à celle de déployer une architecture distribuée. L'utilisation de Spark / Hadoop s'impose lorsque les données sont déjà  dans cet environnement *mais*, très souvent, l'extraction,  l'échantillonnage, la préparation des données, en vue d'un problème spécifique suffit à se ramener à un volume compatible avec la mémoire d'un ordinateur.

Le principal *objectif* est d'introduire ces outils pour l'usage et les usages d'un statisticien ou maintenant *data scientist*. Ce premier tutoriel se focalise donc sur la préparation des données par la gestion de RDDs.

## 2 Présentation de Spark

### 2.1 Documentation
Les procédures d'installation, les concepts et les modes d'utilisation de [Spark](https://spark.apache.org/) sont explicités dans la [documentation](https://spark.apache.org/docs/latest/) en ligne et dans le livre de Karau et al. (2015)\. 

### 2.2 Installation
L'installation de Spark nécessite, entre autres, celle du gestionnaire de projet [Maven](https://maven.apache.org/) et d'une version récente de Java. Cette opération qui est relativement complexe, surtout sous Windows, n'est pas détaillée ici mais consultable dans les documentations citées.  Nous considérons que cela ne fait pas partie des compétences incontournables d'un statisticien qui se focalise sur les méthodes d'analyse / valorisation des données. Spark est donc supposé installé, accessible par l'intermédiaire de l'API *pyspark*. 

### 2.3 Utilisation
Spark propose quatre types ou modes d'exécution:
- *Standalone local* s'exécute dans un processus de machine virtuelle java sur un poste. C'est le mode utilisé pour tester un programme sur un petit ensemble de données et sur un poste de travail.
- *Standalone cluster* est le cadre pour gérer en interne l'ordonnancement des tâches sur un cluster.
- [*MESOS*](http://mesos.apache.org/) pour utiliser un cluster gérer par ces ressources du projet Apache.
- [*YARN*](http://hadoop.apache.org/docs/current/hadoop-yarn/hadoop-yarn-site/YARN.html) même chose pour un cluster (*e.g* AWS) utilisant les fonctionnalités MapReduce de Hadoop.

Spark exécute un programme en Java, Scala ou Python comme un langage interprété, à l'aide d'une console interactive, par le biais d'un notebook ou calepin IPython ou encore dans les environnements (nuages) loués comme par exemple à Amazon's EC2 (*Elastic Cloud Compute*). Pentreath (2015) explicite sur  un exemple le lancement d'un cluster Spark dans l'environnement EC2 d'Amazon.

### 2.4 Configuration
L'environnement utilisé est décrit par la commande `SparkContext` initialisant un objet `SparkConf` qui définit la configuration utilisée comme par exemple l'URL du noeud "maître" (*driver*) du cluster de calcul utilisé, le nombre de noeuds "exécuteurs" ou *workers*, leur espace mémoire réservé à chacun dans le cas de machines virtuelles. Chacun des exécuteurs doit avoir accès à l'insatallation de Python, ses librairies ainsi qu'à l'espace de sctockage où sont distribuées les RDDs.

En utilisation sur un poste seul, cet environnement ne pose pas de problèmes de droits d'accès et est prédéfini à l'installation de Spark.


### 2.5 Programme "jouet"
Charger ce [calepin]() dans le répertoire de travail puis l'ouvrir dans l'environnement [*Jupyter*](http://jupyter.org/). Le fichier texte [`HistorCommande.csv`](https://www.math.univ-toulouse.fr/~besse/Wikistat/data/HistorCommande.csv) contient une liste de "commandes":

    Albert,cafe,3.27
    Albert,chocolat,2.45
    Julie,coca,3.20
    Dominique,tarte,1.50
    Paul,cassoulet,5.40

A l'INSAT, le noyau Jupyter de PySpark est lancé sous Linux dans un navigateur (*firefox*) par l'exécution dans un terminal de la commande:

    pyspark-1.6.2_notebook.sh &
Exécuter les cellules ci-dessous; les principaux objets seront détaillés ensuite.

In [ ]:
# Chargement du fichier
#Renseignez ici le dossier ou vous souhaitez stocker le fichier téléchargé.
DATA_PATH="" 
# Python 2
#import urllib
#f = urllib.urlretrieve("http://www.math.univ-toulouse.fr/~besse/Wikistat/data/HistorCommande.csv","HistorCommande.csv")
#Python 3
import urllib.request
f = urllib.request.urlretrieve("http://www.math.univ-toulouse.fr/~besse/Wikistat/data/HistorCommande.csv",DATA_PATH+"HistorCommande.csv")

In [ ]:
# Lecture et distribution du fichier en RDD
data = sc.textFile(DATA_PATH+"HistorCommande.csv").map(lambda line: 
        line.split(",")).map(lambda record: (record[0], record[1], record[2]))
# Pas d'exécution tant qu'une action n'est pas requise
# nombre total de commandes
NbCommande=data.count()
print("Nb de commandes: %d" % NbCommande)

In [ ]:
#Visualiser le contenu de la RDD
data.take(5)

In [ ]:
# Nombre de clients uniques
ClientUnique = data.map(lambda record: record[0]).distinct().count()
print("Nb clients: %d" % ClientUnique)

In [ ]:
# Total des commandes
TotalCom = data.map(lambda record: float(record[2])).sum()
print("Total des commandes: %2.2f" % TotalCom)

In [ ]:
# Produit le plus commandé
produits = data.map(lambda record: (record[1], 1.0)).reduceByKey(lambda a, b: a + b).collect()
plusFreq = sorted(produits, key=lambda x: x[1], reverse=True)[0]
print("Produit le plus populaire: %s avec %d commandes" %(plusFreq[0],plusFreq[1]))

Le livre de réféfrence est  *Learning Spark* (2015) par  Holden Karau, Andy Konwinski, Patrick Wendell, et Matei Zaharia.  

## 3. Gestion élémentaire de RDDs
La suite du tutoriel s'inspire de ceux proposés par [J. A. Dianes](https://github.com/jadianes/spark-py-notebooks) et utilise les données d'un concours: [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) concernant près de 9M d'interactions dans un réseau. Elles sont décrites en détail [ici](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names). L'objectif est d'apprendre à détecter des intrusions dans un réseau à partir d'un ensemble de variables ou *features* déjà calculées sur chaque transaction ou interaction avec le réseau.

Un sous-échantillon est chargé localement.

In [ ]:
#Python 2
#f = urllib.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz",DATA_PATH+"kddcup.data_10_percent.gz")
#Python 3
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz",DATA_PATH+"kddcup.data_10_percent.gz")

### 3.1 Créer une RDD à partir du fichier
Spark gère directement un fichier compressé.  

In [ ]:
data_file = DATA_PATH+"/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

Aucune exécution tant qu'une action n'est pas lancée pour, par exemple, vérifier que le bon nombre de lignes sont lues.

In [ ]:
raw_data.count()

 Ou en vérifiant le contenue des premières lignes.

In [ ]:
raw_data.take(5)

Autre exemple : créer un RDD à partir d'une liste avec la transformation `parallelize` 

In [ ]:
a = range(100)
data = sc.parallelize(a)

Avant d'exécuter les mêmes actions

In [ ]:
data.count()

In [ ]:
data.take(5)

***N.B.*** Bien faire la distinction entre une *transformation*, déclarative, et une *action* qui s'exécute et provoque l'exécution des transformations précédemment définies. Une transformation est une instruction *paresseuse* (*lazy*) qui n'est exécutée que lorsque cela est nécessaire.

### 3.2 Filtrer une RDD
Une fonction logique est évaluée afin d'extraire les bons éléments d'une RDD pour en créer une nouvelle. Dans l'exemple ci-dessous, il s'agit de compter le nombre de transactions "normales". La commande suivante déclare la transformation:

In [ ]:
normal_raw_data = raw_data.filter(lambda x: 'normal.' in x)

qui est ensuite exécuté par l'action `count`:

In [ ]:
from time import time
t0 = time()
normal_count = normal_raw_data.count()
tt = time() - t0
print("Il y a {}  interactions 'normales'".format(normal_count))
print("Calcul réalisé en {} secondes".format(round(tt,3)))

### 3.3 Transformer une RDD: transformation *map*
Comme précédemmnet, la fonction `lambda` de python est utilisée pour être appliquée dans une transformation *map* à chaque élément de la table. Il s'agit d'extraire chaque champ avec le séparateur "," du format .csv. La première ligne de la RDD créée est ensuite affichée.

In [ ]:
from pprint import pprint
csv_data = raw_data.map(lambda x: x.split(","))
t0 = time()
head_rows = csv_data.take(5)
tt = time() - t0
print("Parse completed in {} seconds".format(round(tt,3)))
pprint(head_rows[0])

La commande suivante prend plus de temps car elle est exécutée à plus de lignes.

In [ ]:
t0 = time()
head_rows = csv_data.take(100000)
tt = time() - t0
print("Parse completed in {} seconds".format(round(tt,3)))

### 3.4 *Map* d'une fonction prédéfine
La transformation *map* applique une fonction prédéfinie à toute la RDD. L'objectif est de définir ci-dessous l'option transaction "normale" comme "valeur de clef" associée à une valeur qui est la liste de tous les autres éléments de la ligne.

In [ ]:
# Fontion qui sépare les champ (elems=valeur) et extrait la 41ème = clef.
def parse_interaction(line):
    elems = line.split(",")
    tag = elems[41]
    return (tag, elems)
# Affichage des 5 premiers
key_csv_data = raw_data.map(parse_interaction)
head_rows = key_csv_data.take(5)
pprint(head_rows[0])

Il est ensuite possible de dénombrer par valeur de clef.

### 3.5 L'action `collect`
En plus des actions `count`et `take`, celle `collect` a pour effet de charger en mémoire tous les éléments d'une RDD. A manier avec prudence pour éviter des dépassements de mémoire.

In [ ]:
t0 = time()
all_raw_data = raw_data.collect()
tt = time() - t0
print("Data collected in {} seconds".format(round(tt,3)))

Cela prend plus de temps pour collecter tous les fragments détenus par les exécuteurs et les regrouper (*reduce*) ensemble. 

Le dernier exemple combine les fonctions précédentes pour collecter toutes les interactions normales comme des paires (clef, valeur).

In [ ]:
# get data from file
data_file = DATA_PATH+"/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

# parse into key-value pairs
key_csv_data = raw_data.map(parse_interaction)

# filter normal key interactions
normal_key_interactions = key_csv_data.filter(lambda x: x[0] == "normal.")

# collect all
t0 = time()
all_normal = normal_key_interactions.collect()
tt = time() - t0
normal_count = len(all_normal)
print("Data collected in {} seconds".format(round(tt,3)))
print("There are {} 'normal' interactions".format(normal_count))

Cette procédure est plus longue que la simple application de l'action `count`. 

## 4 Opération sur des RDDs
### 4.1 Opérations ensemblistes
Des opérateurs ensemblistes: `subtract`, `distinct`, et `cartesian`, sont des transformations qui s'appliquent à des RDDs de même type.

In [ ]:
# Relecture des données
data_file = DATA_PATH+"/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

#### 4.1.1 `Substract`
Obtenir les intrusions ou attaques en retranchant l'ensemble des ineractions "normales".

In [ ]:
# Interactions normales comme précédemment
normal_raw_data = raw_data.filter(lambda x: "normal." in x)

In [ ]:
# Ensemble des intrusions
attack_raw_data = raw_data.subtract(normal_raw_data)

In [ ]:
from time import time

# count all
t0 = time()
raw_data_count = raw_data.count()
tt = time() - t0
print("All count in {} secs".format(round(tt,3)))

In [ ]:
# count normal
t0 = time()
normal_raw_data_count = normal_raw_data.count()
tt = time() - t0
print("Normal count in {} secs".format(round(tt,3)))

In [ ]:
# count attacks
t0 = time()
attack_raw_data_count = attack_raw_data.count()
tt = time() - t0
print("Attack count in {} secs".format(round(tt,3)))

In [ ]:
print("Il y a {} interactions normales and {} attaques, \
pour un total de {} interactions".format(normal_raw_data_count,attack_raw_data_count,raw_data_count))

Le résultat fournit deux RDDs.

#### 4.1.2 `cartesian`
Le produit cartésien retourne toutes les paires possibles entre éléments de deux RDDS. C'est utilisé pour générer toutes les paires entre "service" (2ème colonne) et "protocole" (3ème colone). 

La transformation `distinct` permet de lister les valeurs possibles (uniques) de chaque variable qualitative. 

In [ ]:
# POur le protocole
csv_data = raw_data.map(lambda x: x.split(","))
protocols = csv_data.map(lambda x: x[1]).distinct()
protocols.collect()

In [ ]:
# Pour le service
services = csv_data.map(lambda x: x[2]).distinct()
services.collect()

Le produit cartésien fournit tous les couples possibles.

In [ ]:
product = protocols.cartesian(services).collect()
print("Il y a {} combinaisons de protocol X service".format(len(product)))

A utiliser avec prudence si les RDDs sont volumineuses avec beaucoup de modalités. 

### 4.2 Aggrégations de RDDs

D'autres manipulations ou agrégations sont produites par des actions `reduce`, `fold`, et la dernière: `aggregate` qui est une forme de généralisation.

#### 4.2.1 `reduce`

In [ ]:
# Relecture des données
data_file = DATA_PATH+"/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

Les actions `fold` et `reduce`prennent une fonction comme argument (par exemple "+") qui est appliquée à deux éléments d'une RDD; `fold` diffère de `reduce` car nécessite une initialisation préliminaire (par exemple 0  pour "+").

L'exemple calcule les sommes des temps d'interactions par statut: normales ou non.

In [ ]:
# parse data
csv_data = raw_data.map(lambda x: x.split(","))

# Séparation en deux RDDs
normal_csv_data = csv_data.filter(lambda x: x[41]=="normal.")
attack_csv_data = csv_data.filter(lambda x: x[41]!="normal.")

Les fonctions transférées à `reduce` envoie et retourne des RDDs de même type sous la forme d'une nouvelle RDD à .créer

In [ ]:
# Création de ces RDDs avec les durées
normal_duration_data = normal_csv_data.map(lambda x: int(x[0]))
attack_duration_data = attack_csv_data.map(lambda x: int(x[0]))

La transformation `reduce` appliquée aux deux nouvelles RDDs.

In [ ]:
total_normal_duration = normal_duration_data.reduce(lambda x, y: x + y)
total_attack_duration = attack_duration_data.reduce(lambda x, y: x + y)

print("Total duration for 'normal' interactions is {}".\
    format(total_normal_duration))
print("Total duration for 'attack' interactions is {}".\
    format(total_attack_duration))

Associée à `count` pour calculer la moyenne.

In [ ]:
normal_count = normal_duration_data.count()
attack_count = attack_duration_data.count()

print("Mean duration for 'normal' interactions is {}".\
    format(round(total_normal_duration/float(normal_count),3)))
print("Mean duration for 'attack' interactions is {}".\
    format(round(total_attack_duration/float(attack_count),3)))

Donne une première discrimination simpliste entre interactions.

#### 4.2.2 `aggregate`

L'action `aggregate` relaxe la contrainte de gérer des RDDs de même type mais nécessite l'initialisation comme `fold`. Deux fonctions sont définies, la première de la RDD avec l'accumulateur. La deuxième fusionne deux accumulateurs. L'objectif reste de calculer les moyennes des durées par statut.


In [ ]:
normal_sum_count = normal_duration_data.aggregate((0,0), # valeurs initiales à 0
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # Somme des durées et cumul des interactions
    (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])) # cumul des accumualteurs
    )
print("Durée moyenne des interactions normales {}".\
    format(round(normal_sum_count[0]/float(normal_sum_count[1]),3)))

Le premier accumulateur somme les durées tandis que le 2ème compte le nombre d'interactions. Combiner un accumlateur avec une RDD consiste à sommet la valeur et incrémenter le comptage. Combiner deux accumulateurs est une simple somme par paire.

Même chose avec les attaques.

In [ ]:
attack_sum_count = attack_duration_data.aggregate(
    (0,0), # the initial value
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # combine value with acc
    (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])) # combine accumulators
    )

print("Durée moyenne des interactions agressives {}".\
    format(round(attack_sum_count[0]/float(attack_sum_count[1]),3)))

### 4.3 Combiner par clef

Spark fournit des fonctions pour gérer des paires (clef, valeur), bases du *mapreduce*. Elles exécutent des agrégations et autres traitements par clef.

Ces fonctions permettent d'explorer efficacement différents croisement avec le type d'interaction.

In [ ]:
# relecture des données
data_file = DATA_PATH+"/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

Il s'agit donc, pour explorer, de croiser le type d'interaction avec d'autres variables comme la durée. Ceci est initié par la création de RDDs où la clef est le type d'interaction tandis que la liste des autres champs est la valeur. 

La création des RDDs de paires (clef, valeur) est obtenue par l'applicaiton d'une transformation *map*.

In [ ]:
csv_data = raw_data.map(lambda x: x.split(","))
key_value_data = csv_data.map(lambda x: (x[41], x)) # x[41] contient le type normal ou non

Affichage du premier élément.

In [ ]:
key_value_data.take(1)

#### 4.3.1 Aggrégation de RDDs par clef/valeur

Spark fournit des fonctions spécifiques adaptées aux paires (clef, valeurs). 

Ainsi la transformation `reduceByKey` est utilisée pour calculer la durée par type d'interaction.


In [ ]:
key_value_duration = csv_data.map(lambda x: (x[41], float(x[0]))) 
durations_by_key = key_value_duration.reduceByKey(lambda x, y: x + y)

durations_by_key.collect()

Le comptage est aussi opéré par clef.

In [ ]:
counts_by_key = key_value_data.countByKey()
counts_by_key

#### 4.3.2 `combineByKey`
C'est la fonction d'aggrégation par clef la plus générale et est utilisée par la plupart des autres fonctions de combinaison par clef. Comme pour la transformation `aggregate` les valeurs en retour ne sont pas nécessairement du même type que les données en entrée. 

Toujours pour calculer la moyenne des durées par type:

In [ ]:
sum_counts = key_value_duration.combineByKey(
    (lambda x: (x, 1)), # valeur initiale x and compteur 1
    (lambda acc, value: (acc[0]+value, acc[1]+1)), # Combiner une paire  avec une paires d'accumulateurs (somme et incrément)
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) # combinaison des accumulateurs
     )
sum_counts.collectAsMap()

Les divisions produisent les moyennes.

In [ ]:
duration_means_by_type = sum_counts.map(lambda lambda_args: (lambda_args[0], round(lambda_args[1][0]/lambda_args[1][1],3))).collectAsMap()

#Print them sorted
for tag in sorted(duration_means_by_type, key=duration_means_by_type.get, reverse=True):
    print(tag, duration_means_by_type[tag])